In [0]:
# 変数（前ステップと統一）
catalog_name = "users"
schema = "yukiteru_koide"

# スキーマ切り替え
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema}")

In [0]:
# Bronzeテーブル読み込み
df_transactions = spark.table(f"{catalog_name}.{schema}.transactions")
df_customers = spark.table(f"{catalog_name}.{schema}.customers")
df_products = spark.table(f"{catalog_name}.{schema}.products")
df_stores = spark.table(f"{catalog_name}.{schema}.stores")

# 結合と整形
from pyspark.sql.functions import to_date, col

df_silver = (
    df_transactions
    .withColumn("transaction_date", to_date(col("transaction_date")))
    .filter(col("quantity") > 0)
    .join(df_customers, "customer_id", "left")
    .join(df_products, "product_id", "left")
    .join(df_stores, "store_id", "left")
)

# Silverテーブルとして保存
df_silver.write.format("delta").mode("overwrite").saveAsTable("transactions_enriched")
